# Imports

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

from env import get_db_url

# Acquire & Summarize
Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [2]:
url = get_db_url("zillow")

In [3]:
sql = """
SELECT
Z.parcelid,
Z.basementsqft,
Z.bathroomcnt,
Z.bedroomcnt,
Z.calculatedbathnbr,
Z.finishedfloor1squarefeet,
Z.calculatedfinishedsquarefeet,
Z.finishedsquarefeet12,
Z.finishedsquarefeet13,
Z.finishedsquarefeet15,
Z.finishedsquarefeet50,
Z.finishedsquarefeet6,
Z.fips,
Z.fireplacecnt,
Z.fullbathcnt,
Z.garagecarcnt,
Z.garagetotalsqft,
Z.hashottuborspa,
Z.latitude,
Z.longitude,
Z.lotsizesquarefeet,
Z.poolcnt,
Z.poolsizesum,
Z.propertycountylandusecode,
Z.propertyzoningdesc,
Z.regionidcity,
Z.regionidcounty,
Z.regionidneighborhood,
Z.regionidzip,
Z.roomcnt,
Z.threequarterbathnbr,
Z.unitcnt,
Z.yardbuildingsqft17,
Z.yardbuildingsqft26,
Z.yearbuilt,
Z.numberofstories,
Z.fireplaceflag,
Z.structuretaxvaluedollarcnt,
Z.taxvaluedollarcnt,
Z.assessmentyear,
Z.landtaxvaluedollarcnt,
Z.taxamount,
Z.taxdelinquencyflag,
Z.taxdelinquencyyear,
Z.censustractandblock,
unique_properties.logerror,
unique_properties.transactiondate,
plt.propertylandusedesc,
st.storydesc,
ct.typeconstructiondesc,
act.airconditioningdesc,
bct.buildingclassdesc,
hst.heatingorsystemdesc
FROM 
(SELECT 
p17.parcelid,
logerror,
transactiondate
FROM 
predictions_2017 AS p17
JOIN
(SELECT 
predictions_2017.parcelid,
MAX(transactiondate) AS max_trans_date
FROM predictions_2017
GROUP BY predictions_2017.parcelid) AS pred_agg ON (p17.parcelid=pred_agg.parcelid) AND (pred_agg.max_trans_date=p17.transactiondate)) AS unique_properties
LEFT JOIN properties_2017 AS Z ON (Z.parcelid=unique_properties.parcelid)
LEFT JOIN propertylandusetype AS plt ON (Z.propertylandusetypeid=plt.propertylandusetypeid)
LEFT JOIN storytype AS st ON (Z.storytypeid=st.storytypeid)
LEFT JOIN typeconstructiontype AS ct ON (Z.typeconstructiontypeid=ct.typeconstructiontypeid)
LEFT JOIN airconditioningtype AS act ON (Z.airconditioningtypeid=act.airconditioningtypeid)
LEFT JOIN architecturalstyletype AS ast ON (Z.architecturalstyletypeid=ast.architecturalstyletypeid)
LEFT JOIN buildingclasstype AS bct ON (Z.buildingclasstypeid=bct.buildingclasstypeid)
LEFT JOIN heatingorsystemtype AS hst ON (Z.heatingorsystemtypeid=hst.heatingorsystemtypeid)
WHERE Z.latitude IS NOT NULL AND Z.longitude IS NOT NULL
"""


In [4]:
df = pd.read_sql(sql, url)

In [5]:
df.head()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,...,taxdelinquencyyear,censustractandblock,logerror,transactiondate,propertylandusedesc,storydesc,typeconstructiondesc,airconditioningdesc,buildingclassdesc,heatingorsystemdesc
0,14297519,nan,3.50,4.00,3.50,nan,"3,100.00","3,100.00",nan,nan,...,nan,"60,590,630,072,012.00",0.03,2017-01-01,Single Family Residential,None,None,None,None,None
1,17052889,nan,1.00,2.00,1.00,"1,465.00","1,465.00","1,465.00",nan,nan,...,nan,"61,110,010,023,006.00",0.06,2017-01-01,Single Family Residential,None,None,None,None,None
2,14186244,nan,2.00,3.00,2.00,nan,"1,243.00","1,243.00",nan,nan,...,nan,"60,590,218,022,012.00",0.01,2017-01-01,Single Family Residential,None,None,None,None,None
3,12177905,nan,3.00,4.00,3.00,nan,"2,376.00","2,376.00",nan,nan,...,nan,"60,373,001,001,006.00",-0.10,2017-01-01,Single Family Residential,None,None,None,None,Central
4,10887214,nan,3.00,3.00,3.00,nan,"1,312.00","1,312.00",nan,nan,...,nan,"60,371,236,012,000.00",0.01,2017-01-01,Condominium,None,None,Central,None,Central


In [6]:
df.describe()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,"77,381.00",50.00,"77,381.00","77,381.00","76,772.00","6,023.00","77,185.00","73,749.00",41.00,"3,009.00",...,"17,560.00",172.00,"77,269.00","77,380.00","77,381.00","77,379.00","77,376.00","2,886.00","77,137.00","77,381.00"
mean,"13,007,150.51",679.72,2.30,3.05,2.32,"1,366.51","1,785.22","1,760.52","1,389.85","2,357.79",...,1.43,1.00,"189,314.53","490,134.48","2,016.00","301,095.41","5,995.53",14.09,"60,496,733,646,264.16",0.02
std,"3,481,345.65",689.70,1.00,1.14,0.98,671.31,954.05,934.02,123.53,"1,188.78",...,0.54,0.00,"230,087.42","653,443.97",0.00,"492,596.03","7,622.84",2.19,"1,535,241,981,138.57",0.17
min,"10,711,855.00",38.00,0.00,0.00,1.00,44.00,128.00,128.00,"1,056.00",598.00,...,1.00,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00",-4.66
25%,"11,538,305.00",273.00,2.00,2.00,2.00,955.00,"1,182.00","1,172.00","1,344.00","1,625.00",...,1.00,1.00,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.63",14.00,"60,373,109,005,002.00",-0.02
50%,"12,531,568.00",515.00,2.00,3.00,2.00,"1,257.00","1,543.00","1,523.00","1,440.00","2,094.00",...,1.00,1.00,"136,499.00","358,975.50","2,016.00","203,372.00","4,450.69",15.00,"60,376,032,003,008.00",0.01
75%,"14,211,831.00",796.50,3.00,4.00,3.00,"1,615.00","2,113.00","2,076.00","1,440.00","2,838.00",...,2.00,1.00,"218,787.00","569,001.50","2,016.00","366,796.50","6,927.79",15.00,"60,590,423,251,008.00",0.04
max,"167,689,317.00","3,560.00",18.00,16.00,18.00,"6,912.00","35,640.00","21,929.00","1,560.00","35,640.00",...,6.00,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00",5.26


In [7]:
df.parcelid.nunique()

77381

In [8]:
df.shape

(77381, 53)

In [9]:
df.columns

Index(['parcelid', 'basementsqft', 'bathroomcnt', 'bedroomcnt',
       'calculatedbathnbr', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'propertycountylandusecode', 'propertyzoningdesc', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'threequarterbathnbr', 'unitcnt', 'yardbuildingsqft17',
       'yardbuildingsqft26', 'yearbuilt', 'numberofstories', 'fireplaceflag',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'taxdelinquencyflag',
       'taxdelinquencyyear', 'censustractandblock', 'logerror',
       'transactiondate', 'proper

In [10]:
print("---Info")
print(df.info())

---Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 53 columns):
parcelid                        77381 non-null int64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
calculatedbathnbr               76772 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12            73749 non-null float64
finishedsquarefeet13            41 non-null float64
finishedsquarefeet15            3009 non-null float64
finishedsquarefeet50            6023 non-null float64
finishedsquarefeet6             386 non-null float64
fips                            77381 non-null float64
fireplacecnt                    8276 non-null float64
fullbathcnt                     76772 non-null float64
garagecarcnt                    25474 non-null float64
garagetotalsqft  

In [11]:
def df_summary(df):
    print("---Shape:",df.shape)
    print("---Info")
    print(df.info())
    print("---Describe")
    print(df.describe())

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

attributes = pd.DataFrame(         {'num_rows_missing': df.isnull().sum(),           'pct_rows_missing': df.isna().sum() / len(df),         },         index=df.columns )

In [12]:
def nulls_by_column(df):
    """
    Returns a dataframe that tells you how many rows are null and the percentage that is missing in that column
    """
    attributes = pd.DataFrame({"num_rows_missing": df.isnull().sum(), "pct_row_missing": df.isna().sum()/len(df)}, index=df.columns)
    return attributes

In [13]:
nulls_by_column(df)

,num_rows_missing,pct_row_missing
parcelid,0,0.00
basementsqft,77331,1.00
bathroomcnt,0,0.00
bedroomcnt,0,0.00
calculatedbathnbr,609,0.01
finishedfloor1squarefeet,71358,0.92
calculatedfinishedsquarefeet,196,0.00
finishedsquarefeet12,3632,0.05
finishedsquarefeet13,77340,1.00
finishedsquarefeet15,74372,0.96


**Takeaways**
- Drop all columns with missing rows that are greater than 50% as these will not provide us with any useful data because there is to much missing data

Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [14]:
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing 

In [15]:
df.isnull().sum(axis=1)

0        24
1        21
2        23
3        23
4        21
         ..
77376    23
77377    20
77378    23
77379    24
77380    24
Length: 77381, dtype: int64

In [16]:
all_rows = pd.DataFrame({"num_cols_missing": df.isnull().sum(axis=1), "pct_cols_missing": df.isna().sum(axis=1)/len(df.columns)})

In [17]:
all_rows.num_cols_missing.value_counts()

23    24003
24    17749
22    14752
21     8762
25     4658
20     2661
26     2163
19     1074
18      482
27      323
17      244
28      237
32       79
33       50
16       46
29       30
31       27
30       24
15        6
34        5
35        3
36        3
Name: num_cols_missing, dtype: int64

In [18]:
df.isna().sum(axis=1)/len(df.columns)

0                       0.45
1                       0.40
2                       0.43
3                       0.43
4                       0.40
                ...         
77376                   0.43
77377                   0.38
77378                   0.43
77379                   0.45
77380                   0.45
Length: 77381, dtype: float64

# Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 53 columns):
parcelid                        77381 non-null int64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
calculatedbathnbr               76772 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12            73749 non-null float64
finishedsquarefeet13            41 non-null float64
finishedsquarefeet15            3009 non-null float64
finishedsquarefeet50            6023 non-null float64
finishedsquarefeet6             386 non-null float64
fips                            77381 non-null float64
fireplacecnt                    8276 non-null float64
fullbathcnt                     76772 non-null float64
garagecarcnt                    25474 non-null float64
garagetotalsqft          

In [29]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52320
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [30]:
single_family_df = df[df.propertylandusedesc.isin(["Single Family Residential", "Residential General"])]

In [31]:
single_family_df.bedroomcnt.value_counts(dropna=False)

 3.00    23305
 4.00    15213
 2.00     8317
 5.00     3969
 6.00      634
 1.00      612
 0.00      162
 7.00      106
 8.00       25
 9.00        8
10.00        2
11.00        2
12.00        1
14.00        1
Name: bedroomcnt, dtype: int64

In [32]:
single_family_df.bathroomcnt.value_counts(dropna=False)

 2.00    21841
 3.00    10654
 1.00     9537
 2.50     3932
 4.00     2226
 3.50      916
 1.50      839
 5.00      804
 4.50      686
 6.00      321
 5.50      224
 0.00      146
 7.00       89
 8.00       53
 6.50       47
 7.50       16
 9.00       13
10.00        5
11.00        3
 8.50        3
13.00        1
18.00        1
Name: bathroomcnt, dtype: int64

In [36]:
# Keep just homes with bed and bath count greater than 0
single_family_df = single_family_df[(single_family_df.bathroomcnt > 0) & (single_family_df.bedroomcnt > 0)]

In [37]:
single_family_df.head()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,...,taxdelinquencyyear,censustractandblock,logerror,transactiondate,propertylandusedesc,storydesc,typeconstructiondesc,airconditioningdesc,buildingclassdesc,heatingorsystemdesc
0,14297519,nan,3.50,4.00,3.50,nan,"3,100.00","3,100.00",nan,nan,...,nan,"60,590,630,072,012.00",0.03,2017-01-01,Single Family Residential,None,None,None,None,None
1,17052889,nan,1.00,2.00,1.00,"1,465.00","1,465.00","1,465.00",nan,nan,...,nan,"61,110,010,023,006.00",0.06,2017-01-01,Single Family Residential,None,None,None,None,None
2,14186244,nan,2.00,3.00,2.00,nan,"1,243.00","1,243.00",nan,nan,...,nan,"60,590,218,022,012.00",0.01,2017-01-01,Single Family Residential,None,None,None,None,None
3,12177905,nan,3.00,4.00,3.00,nan,"2,376.00","2,376.00",nan,nan,...,nan,"60,373,001,001,006.00",-0.10,2017-01-01,Single Family Residential,None,None,None,None,Central
6,12095076,nan,3.00,4.00,3.00,nan,"2,962.00","2,962.00",nan,nan,...,nan,"60,374,608,001,014.00",-0.00,2017-01-01,Single Family Residential,None,None,Central,None,Central


In [38]:
single_family_df.unitcnt.value_counts()

1.00    33699
2.00       26
4.00        2
3.00        1
Name: unitcnt, dtype: int64

In [39]:
single_family_df.unitcnt.isnull().sum()

18451

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [40]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    """
    Takes in a df and drops columns and rows based on the percent of values that are missing
    """
    # column threshold for whether we keep or remove the column
    threshold = int(round(prop_required_column*len(df.index),0))
    # axis=1 argument means "column"
    df.dropna(axis=1, thresh=threshold, inplace=True)
    
    threshold = int(round(prop_required_row*len(df.columns),0))
    # axis=0 argument means "rows"
    # drop the row if over the threshold
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [41]:
#make sure it works
handle_missing_values(single_family_df)


,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,propertylandusedesc,heatingorsystemdesc
0,14297519,3.50,4.00,3.50,"3,100.00","3,100.00","6,059.00",3.00,"33,634,931.00","-117,869,207.00",...,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0.03,2017-01-01,Single Family Residential,None
1,17052889,1.00,2.00,1.00,"1,465.00","1,465.00","6,111.00",1.00,"34,449,266.00","-119,281,531.00",...,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",0.06,2017-01-01,Single Family Residential,None
2,14186244,2.00,3.00,2.00,"1,243.00","1,243.00","6,059.00",2.00,"33,886,168.00","-117,823,170.00",...,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",0.01,2017-01-01,Single Family Residential,None
3,12177905,3.00,4.00,3.00,"2,376.00","2,376.00","6,037.00",3.00,"34,245,180.00","-118,240,722.00",...,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",-0.10,2017-01-01,Single Family Residential,Central
6,12095076,3.00,4.00,3.00,"2,962.00","2,962.00","6,037.00",3.00,"34,145,202.00","-118,179,824.00",...,"276,684.00","773,303.00","2,016.00","496,619.00","9,516.26","60,374,608,001,014.00",-0.00,2017-01-01,Single Family Residential,Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77376,11000655,2.00,2.00,2.00,"1,286.00","1,286.00","6,037.00",2.00,"34,245,368.00","-118,282,383.00",...,"70,917.00","354,621.00","2,016.00","283,704.00","4,478.43","60,371,014,003,002.00",0.02,2017-09-20,Single Family Residential,Central
77377,17239384,2.00,4.00,2.00,"1,612.00","1,612.00","6,111.00",2.00,"34,300,140.00","-118,706,327.00",...,"50,683.00","67,205.00","2,016.00","16,522.00","1,107.48","61,110,084,022,016.00",0.01,2017-09-21,Single Family Residential,None
77378,12773139,1.00,3.00,1.00,"1,032.00","1,032.00","6,037.00",1.00,"34,040,895.00","-118,038,169.00",...,"32,797.00","49,546.00","2,016.00","16,749.00",876.43,"60,374,338,022,005.00",0.04,2017-09-21,Single Family Residential,Central
77379,12826780,2.00,3.00,2.00,"1,762.00","1,762.00","6,037.00",2.00,"33,937,685.00","-117,996,709.00",...,"140,000.00","522,000.00","2,016.00","382,000.00","6,317.15","60,375,034,021,020.00",0.01,2017-09-25,Single Family Residential,Central


In [42]:
df_summary(single_family_df)

---Shape: (52178, 29)
---Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 52178 entries, 0 to 77380
Data columns (total 29 columns):
parcelid                        52178 non-null int64
bathroomcnt                     52178 non-null float64
bedroomcnt                      52178 non-null float64
calculatedbathnbr               52163 non-null float64
calculatedfinishedsquarefeet    52171 non-null float64
finishedsquarefeet12            52013 non-null float64
fips                            52178 non-null float64
fullbathcnt                     52163 non-null float64
latitude                        52178 non-null float64
longitude                       52178 non-null float64
lotsizesquarefeet               51823 non-null float64
propertycountylandusecode       52178 non-null object
propertyzoningdesc              33700 non-null object
regionidcity                    51150 non-null float64
regionidcounty                  52178 non-null float64
regionidzip                     52155 no

In [43]:
single_family_df.isnull().sum()

parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
calculatedbathnbr                  15
calculatedfinishedsquarefeet        7
finishedsquarefeet12              165
fips                                0
fullbathcnt                        15
latitude                            0
longitude                           0
lotsizesquarefeet                 355
propertycountylandusecode           0
propertyzoningdesc              18478
regionidcity                     1028
regionidcounty                      0
regionidzip                        23
roomcnt                             0
unitcnt                         18450
yearbuilt                          39
structuretaxvaluedollarcnt         72
taxvaluedollarcnt                   1
assessmentyear                      0
landtaxvaluedollarcnt               1
taxamount                           4
censustractandblock               113
logerror                            0
transactiond

In [45]:
single_family_df.calculatedbathnbr.value_counts(dropna=False)

 2.00    21833
 3.00    10650
 1.00     9519
 2.50     3923
 4.00     2226
 3.50      916
 1.50      832
 5.00      803
 4.50      686
 6.00      321
 5.50      223
 7.00       89
 8.00       53
 6.50       47
 7.50       16
  nan       15
 9.00       13
10.00        5
11.00        3
 8.50        3
13.00        1
18.00        1
Name: calculatedbathnbr, dtype: int64

In [48]:
(single_family_df.calculatedbathnbr == single_family_df.bathroomcnt).sum() / len(single_family_df)

0.9997125225190694

In [49]:
#drop columns
single_family_df = single_family_df.drop(columns=["calculatedbathnbr", "censustractandblock"])

In [51]:
single_family_df.isnull().sum()

parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
calculatedfinishedsquarefeet        7
finishedsquarefeet12              165
fips                                0
fullbathcnt                        15
latitude                            0
longitude                           0
lotsizesquarefeet                 355
propertycountylandusecode           0
propertyzoningdesc              18478
regionidcity                     1028
regionidcounty                      0
regionidzip                        23
roomcnt                             0
unitcnt                         18450
yearbuilt                          39
structuretaxvaluedollarcnt         72
taxvaluedollarcnt                   1
assessmentyear                      0
landtaxvaluedollarcnt               1
taxamount                           4
logerror                            0
transactiondate                     0
propertylandusedesc                 0
heatingorsys

In [52]:
df.heatingorsystemtypeid

AttributeError: 'DataFrame' object has no attribute 'heatingorsystemtypeid'

In [53]:
df.regionidneighborhood

0                        nan
1                        nan
2                        nan
3                        nan
4                 268,548.00
                ...         
77376              27,328.00
77377                    nan
77378                    nan
77379                    nan
77380                    nan
Name: regionidneighborhood, Length: 77381, dtype: float64